In [2]:
# safe divide function to stop zero counts from causing problems. This is zero counts for number of words
# which can happen sometimes when texts are small, texts are empty, or content words are removed from small texts
def safe_divide(a, b): #function calls two arguments. Here it will be a POS count and number of words
    if b != 0: #
        return a/b
    else:
        return 0

In [3]:
!pip install pandas
! python -m spacy download en

import pandas as pd
import re
import spacy

nlp = spacy.load('en_core_web_sm')


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
#import the dataset
text_df = pd.read_excel('/Users/yuchengbao/PycharmProjects/JupyterProject/models/Practice Dataset.xlsx')

In [18]:
# pre-process the text data (text_df)
text_df['input'] = text_df['input'].astype(str) # convert the text input to string type
text_df['input'] = text_df['input'].str.lower() # convert the text to lower case

In [ ]:
# detect the gibberish and cross-check with the human labels
! pip install langdetect
from langdetect import detect_langs

In [36]:
#generate a new column 'gibberish_detected' to store the gibberish detection results
text_df['gibberish_detected'] = 0  # default column

for index, row in text_df.iterrows():
    try:
        langs = detect_langs(row['input'])

        if len(langs) > 0:
            lang = langs[0].lang
            prob = langs[0].prob

            # Mark as gibberish if not English OR English confidence < threshold
            if langs[0].lang != 'en' and langs[0].prob > 0.8:
                text_df.at[index, 'gibberish_detected'] = 1
            else:
                text_df.at[index, 'gibberish_detected'] = 0
    except:
        text_df.at[index, 'gibberish_detected'] = 1  # Mark as gibberish if detection fails

# compare with human labels
text_df['gibberish_match'] = text_df['gibberish_detected'] == text_df['Invalid Response']
# calculate accuracy
accuracy = text_df['gibberish_match'].mean()
print(f'Gibberish detection accuracy: {accuracy:.2%}')

#pull out the Anon.Student.Id, input, Invalid Response, gibberish_detected, gibberish_match columns for review
text_df_quick_review = text_df[['Anon.Student.Id', 'input', 'Invalid Response', 'gibberish_detected', 'gibberish_match']]

Gibberish detection accuracy: 78.06%


In [38]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# True labels (from human-coded data)
y_true = text_df['Invalid Response']
# Predicted labels (from your detection code)
y_pred = text_df['gibberish_detected']

# 1️⃣ Accuracy: How many total predictions were correct
accuracy = accuracy_score(y_true, y_pred)

# 2️⃣ Precision (Of the items flagged as gibberish, how many were actually gibberish), Recall (How many actual gibberish cases were caught), F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# 3️⃣ Full classification report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["Valid", "Gibberish"]))

# 4️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\n=== Confusion Matrix ===")
print(cm)

# 5️⃣ Summary output
print(f"\nAccuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")

=== Classification Report ===
              precision    recall  f1-score   support

       Valid       0.90      0.74      0.81       295
   Gibberish       0.65      0.86      0.74       170

    accuracy                           0.78       465
   macro avg       0.78      0.80      0.78       465
weighted avg       0.81      0.78      0.78       465


=== Confusion Matrix ===
[[217  78]
 [ 24 146]]

Accuracy: 78.06%
Precision: 65.18%
Recall: 85.88%
F1-score: 74.11%


In [22]:
#output the review file
text_df_quick_review.to_excel('/Users/yuchengbao/PycharmProjects/JupyterProject/models/gibberish_detection_review.xlsx', index=False)

In [ ]:
text_data_docs = list(nlp.pipe(text_df.input)) #spacy the texts

In [23]:
# preview the text_df
text_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy (used to be Efficacy Management/ Efficacy Self-Talk),Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost (outside effort cost),Effort,Extrinsic Rewards/Self-Consequating,Environmental Control,...,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Unnamed: 22,gibberish_detected,gibberish_match
0,stu_NWRXR49435,that i will awaly do better then last time,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,0.0,True
1,stu_GDLLQ68531,no,0,0,0,0,0,0,0,0.0,...,0,0,0,1.0,0,0.0,1,NaN,1.0,False
2,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1.0,True
3,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1.0,True
4,stu_KAIDW62100,". jjjnnk.nj..mi,.mk.ki.kii..l.kin.",1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,sch_NQNWK22810,i dont know i jjust worked hard,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,1,0.0,1,NaN,0.0,True
461,sch_HTPUV14121,i didnt,0,0,0,0,0,0,0,0.0,...,0,0,0,1.0,0,0.0,1,NaN,0.0,True
462,sch_NQNWK22810,i dont want to fail so i kept telling myself t...,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,0.0,True
463,sch_HTPUV14121,",ndcfk,sdj",1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1.0,True
